This is my ongoing final year project for my degree in Data Science and Computing. The project is to experiment with different LLMs for the task of multiple choice question answering in the medical domain. The dataset used is the MedMCQA dataset from hugging face hub. The model is fine-tuned using tensorflow and keras. The model is trained on google colab and the model is saved in google drive. The model is evaluated on the test dataset to measure its accuracy. The project is expected to be done by the end of April 2024. 

# Install & Import packages

In [ ]:
!pip install transformers
!pip install datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 74.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00


In [ ]:
import numpy as np
import tensorflow as tf
import os
import math

# Download dataset

Download medmcqa dataset from hugging face hub


In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset("medmcqa")

Generating train split:   0%|          | 0/182822 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6150 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4183 [00:00<?, ? examples/s]

Explore dataset

In [ ]:
raw_train_dataset = raw_datasets["train"]

print("Sample training data: ")
raw_train_dataset[0]

Sample training data: 


{'id': 'e9ad821a-c438-4965-9f77-760819dfa155',
 'question': 'Chronic urethral obstruction due to benign prismatic hyperplasia can lead to the following change in kidney parenchyma',
 'opa': 'Hyperplasia',
 'opb': 'Hyperophy',
 'opc': 'Atrophy',
 'opd': 'Dyplasia',
 'cop': 2,
 'choice_type': 'single',
 'exp': 'Chronic urethral obstruction because of urinary calculi, prostatic hyperophy, tumors, normal pregnancy, tumors, uterine prolapse or functional disorders cause hydronephrosis which by definition is used to describe dilatation of renal pelvis and calculus associated with progressive atrophy of the kidney due to obstruction to the outflow of urine Refer Robbins 7yh/9,1012,9/e. P950',
 'subject_name': 'Anatomy',
 'topic_name': 'Urinary tract'}

In [ ]:
raw_train_dataset.features

{'id': Value(dtype='string', id=None),
 'question': Value(dtype='string', id=None),
 'opa': Value(dtype='string', id=None),
 'opb': Value(dtype='string', id=None),
 'opc': Value(dtype='string', id=None),
 'opd': Value(dtype='string', id=None),
 'cop': ClassLabel(names=['a', 'b', 'c', 'd'], id=None),
 'choice_type': Value(dtype='string', id=None),
 'exp': Value(dtype='string', id=None),
 'subject_name': Value(dtype='string', id=None),
 'topic_name': Value(dtype='string', id=None)}

# Prepare the dataset for training

As choices are stored in different features, we need to concatinate them with their questions as text input into the training model

In [ ]:
def concat_questions_with_answers(example):
  example["question"] = """
  {question}.

  Please select one of the following questions:
  A. {opa}
  B. {opb}
  C. {opc}
  D. {opd}

  The answer is
  """.format(
      question=example["question"],
      opa=example["opa"],
      opb=example["opb"],
      opc=example["opc"],
      opd=example["opd"]
  )

  return example

raw_datasets = raw_datasets.map(concat_questions_with_answers)

Map:   0%|          | 0/182822 [00:00<?, ? examples/s]

Map:   0%|          | 0/6150 [00:00<?, ? examples/s]

Map:   0%|          | 0/4183 [00:00<?, ? examples/s]

In [ ]:
raw_datasets["train"][0]

{'id': 'e9ad821a-c438-4965-9f77-760819dfa155',
 'question': 'Chronic urethral obstruction due to benign prismatic hyperplasia can lead to the following change in kidney parenchyma. Please select one of the following questions:\n  A. Hyperplasia B. Hyperophy C. Atrophy D. Dyplasia\n  ',
 'opa': 'Hyperplasia',
 'opb': 'Hyperophy',
 'opc': 'Atrophy',
 'opd': 'Dyplasia',
 'cop': 2,
 'choice_type': 'single',
 'exp': 'Chronic urethral obstruction because of urinary calculi, prostatic hyperophy, tumors, normal pregnancy, tumors, uterine prolapse or functional disorders cause hydronephrosis which by definition is used to describe dilatation of renal pelvis and calculus associated with progressive atrophy of the kidney due to obstruction to the outflow of urine Refer Robbins 7yh/9,1012,9/e. P950',
 'subject_name': 'Anatomy',
 'topic_name': 'Urinary tract'}

Explore data types of each feature

In [ ]:
raw_datasets["train"].features

{'id': Value(dtype='string', id=None),
 'question': Value(dtype='string', id=None),
 'opa': Value(dtype='string', id=None),
 'opb': Value(dtype='string', id=None),
 'opc': Value(dtype='string', id=None),
 'opd': Value(dtype='string', id=None),
 'cop': ClassLabel(names=['a', 'b', 'c', 'd'], id=None),
 'choice_type': Value(dtype='string', id=None),
 'exp': Value(dtype='string', id=None),
 'subject_name': Value(dtype='string', id=None),
 'topic_name': Value(dtype='string', id=None)}

Choose the correct tokenizer for the training model

In [ ]:
checkpoint = "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext"

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenized_questions = tokenizer(raw_datasets["train"]["question"])

In [ ]:
tokenized_questions

Tokenize the raw datasets

In [ ]:
def tokenize_function(example):
  return tokenizer(example["question"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

Padding text to a certain length

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

Split tokenized datasets into train, validation & test datasets

In [ ]:
tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["cop"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8
)

tf_validation_dataset = tokenized_datasets["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["cop"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8
)

tf_test_dataset = tokenized_datasets["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["cop"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8
)

# For loading saved models

In [ ]:
path = 'drive/MyDrive/medmcqa'
os.listdir(path)

['config.json',
 'tf_model.h5',
 'bert-cp',
 'cp-0001.ckpt.index',
 'cp-0001.ckpt.data-00000-of-00001',
 'checkpoint']

In [ ]:
saved_model = model.from_pretrained(path)
saved_mode.summary()

KeyboardInterrupt: ignored

# Build training model

In [ ]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=4, from_pt=True)

Configure learning rate for faster training

In [ ]:
from tensorflow.keras.optimizers.schedules import PolynomialDecay

batch_size = 8
num_epochs = 3

num_train_steps = len(tf_train_dataset) * num_epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5,
    end_learning_rate=0.0,
    decay_steps=num_train_steps
)

from tensorflow.keras.optimizers import Adam

opt = Adam(learning_rate=lr_scheduler)

Configure persistent storage of model checkpoint

In [ ]:
checkpoint_path = "drive/MyDrive/medmcqa/cp-{epoch:04d}.ckpt"

n_batches = len(tf_train_dataset) / batch_size
n_batches = math.ceil(n_batches)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5*n_batches
)

Configure the model for training

In [ ]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy

model.compile(
    optimizer=opt,
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

In [ ]:
# load pre-saved model
model.from_pretrained("drive/MyDrive/medmcqa/bert-cp")

Some layers from the model checkpoint at drive/MyDrive/medmcqa/bert-cp were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at drive/MyDrive/medmcqa/bert-cp.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [ ]:
model.fit(
    tf_train_dataset,
    callbacks=[cp_callback],
    validation_data=tf_validation_dataset,
    epochs=1
)

Using bos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using bos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using bos_token, but it is not set yet.


    1/22853 [..............................] - ETA: 332:38:35 - loss: 1.3640 - accuracy: 0.3750

Using eos_token, but it is not set yet.


    2/22853 [..............................] - ETA: 67:27:52 - loss: 1.4252 - accuracy: 0.2500 

Using bos_token, but it is not set yet.


...

  140/22853 [..............................] - ETA: 42:40:04 - loss: 1.3932 - accuracy: 0.2875

Using bos_token, but it is not set yet.


  141/22853 [..............................] - ETA: 42:49:10 - loss: 1.3926 - accuracy: 0.2881

Using eos_token, but it is not set yet.


14284/22853 [=================>............] - ETA: 32:13 - loss: 1.3848 - accuracy: 0.2804
Epoch 1: saving model to drive/MyDrive/medmcqa/cp-0001.ckpt
22853/22853 [==============================] - 4659s 202ms/step - loss: 1.3832 - accuracy: 0.2854 - val_loss: 1.3695 - val_accuracy: 0.3223


Save the model

In [ ]:
model.save_pretrained("drive/MyDrive/medmcqa/bert-cp", from_pt=True)

# Evaluate the model on test dataset

In [ ]:
loss, acc = model.evaluate(tf_test_dataset)
print("Model accuracy on test dataset: {:5.2f}%".format(acc * 100))

 93/769 [==>...........................] - ETA: 34s - loss: nan - accuracy: 0.0000e+00

KeyboardInterrupt: ignored

In [ ]:
preds = model.predict(tf_validation_dataset)["logits"]